In [1]:
import cv2
from PIL import Image, ImageOps
import os
import git
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from tensorflow.keras.preprocessing.image import img_to_array, load_img, array_to_img
import subprocess
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.stats import randint
import numpy as np
from sklearn.svm import SVC

def make_dataset(size=28):
    repo_url = "https://github.com/pongsapaks/Thai-handwrittingnumberproject.git"
    repo_dir = "Thai-handwrittingnumberproject"
    subprocess.run(["git", "clone", repo_url, repo_dir])

    image_dir = os.path.join(repo_dir, "raw")
    image_files = []
    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith(".png"):
                image_files.append(os.path.join(root, file))

    print("Total image files:", len(image_files))

    X = []
    Y = []

    for image_path in image_files:
        img = cv2.imread(image_path)
        img = Image.open(image_path).convert("L")
        img = ImageOps.invert(img)
        img = img.resize((size, size))
        label = os.path.basename(os.path.dirname(image_path))
        x = np.array(img)
        X.append(x)
        Y.append(label)

    X = np.asarray(X)
    Y = np.asarray(Y)

    reshaped_X = X.reshape((X.shape[0], -1))
    Ydf = pd.DataFrame(Y)
    Xdf = pd.DataFrame(reshaped_X)

    X_mean = Xdf.mean()
    X_std = Xdf.std()
    Z = (Xdf - X_mean) / X_std
    Z = Z.fillna(0)

    with open("std_params.pkl", "wb") as f:
        pickle.dump((X_mean, X_std), f)

    pca = PCA(n_components=0.75)
    pca.fit(Z)
    X_pca = pca.transform(Z)

    X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=42)

    sv = SVC(C=10, gamma=0.001, kernel='rbf')
    sv.fit(X_train, y_train)
    pred = sv.predict(X_test)
    accuracy = accuracy_score(y_test, pred)
    print(f"Model accuracy: {accuracy}")

    with open("model.pkl", "wb") as f:
        pickle.dump(sv, f)
    with open("pca.pkl", "wb") as f:
        pickle.dump(pca, f)
make_dataset()


Total image files: 1795
Model accuracy: 0.7632311977715878


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

def preprocess_image(image_path, size=28):
    img = Image.open(image_path).convert("L")
    img = ImageOps.invert(img)
    img = img.resize((size, size))
    img_arr = np.array(img)
    return img_arr

def predict_image(image_path, model_path="model.pkl", pca_path="pca.pkl", std_params_path="std_params.pkl", size=28):
    # Load the model, pca, and standardization parameters from files
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    with open(pca_path, 'rb') as f:
        pca = pickle.load(f)
    with open(std_params_path, 'rb') as f:
        X_mean, X_std = pickle.load(f)

    # Preprocess the image
    image = preprocess_image(image_path, size=size)

    # Flatten and standardize the image
    reshaped_image = image.reshape((1, -1))
    # Avoid division by zero by adding a small constant to X_std
    epsilon = 1e-8
    standardized_image = (reshaped_image - np.array(X_mean).reshape(1,-1)) / (np.array(X_std).reshape(1,-1) + epsilon)

    if np.isnan(standardized_image).any():
        print("standardized_image still contains NaN values!")

    # Apply PCA
    transformed_image = pca.transform(standardized_image)

    # Predict the label
    prediction = model.predict(transformed_image)

    return prediction

In [3]:
from PIL import Image, ImageOps
import numpy as np

image_path = r'C:\Users\Karuntarat\OneDrive\1st year\1st Year, 2nd Term\DADs6003 - ML\Thai HandWriting\Thai-handwrittingnumberproject\3d0e872f-4def-4d84-823f-9c5606f7f672.png'  # replace with the actual path to your image
prediction = predict_image(image_path)
print("The predicted label is:", prediction[0])

The predicted label is: 6


In [6]:
from flask import Flask, request
from PIL import Image
from io import BytesIO
import base64
import dash
from dash import dcc, html
from dash_canvas import DashCanvas
import dash_bootstrap_components as dbc
import requests

# Initialize the Flask app
server = Flask(__name__)

@server.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    image_data = data['image']
    image_data = base64.b64decode(image_data.split(',')[1])
    image = Image.open(BytesIO(image_data))

    # preprocess and predict
    prediction = predict_image(image) # Modify this line to suit the preprocessing and prediction in your case
    return str(prediction)

# Initialize the Dash app
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets) # server=server connects Dash to Flask

canvas_width = 500

app.layout = html.Div([
    DashCanvas(id='canvas',
               lineWidth=5,
               width=canvas_width,
               ),
    html.Button('Predict', id='button_predict', n_clicks=0),
    html.Div(id='prediction')
])

@app.callback(
    dash.dependencies.Output('prediction', 'children'),
    [dash.dependencies.Input('button_predict', 'n_clicks')],
    [dash.dependencies.State('canvas', 'json_data')]
)
def update_output(n_clicks, json_data):
    if n_clicks > 0:
        image_data = json_data['image']
        response = requests.post('http://localhost:5000/predict', json={'image': image_data})
        return 'Predicted number: {}'.format(response.text)

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

c:\Users\Karuntarat\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import gradio as gr
import cv2
import numpy as np
import joblib
from PIL import ImageOps

# Load the pre-trained model and PCA
sv = joblib.load("model.pkl")
pca = joblib.load("pca.pkl")

# Load the mean and standard deviation for feature scaling
X_mean = np.load("X_mean.npy")
X_std = np.load("X_std.npy")

def recognize_handwriting(img):
    try:
        # Preprocess the user input image
        img_gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
        img_resized = cv2.resize(img_gray, (28, 28))
        x = img_resized.flatten().reshape(1, -1)
        x_scaled = (x - X_mean) / X_std
        x_pca = pca.transform(x_scaled)
        prediction = sv.predict(x_pca)[0]

        return f"The predicted digit is: {prediction}"
    except Exception as e:
        print(e)  # Optional: print the error message to the console
        return "Character not recognized"

iface = gr.Interface(fn=recognize_handwriting, inputs="sketchpad", outputs="label")
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


OpenCV(4.7.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<1,-1,-1>,struct cv::impl::A0x981fb336::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1



In [4]:
import gradio as gr
import cv2
import numpy as np
import joblib
from PIL import ImageOps

# Load the pre-trained model and PCA
sv = joblib.load("model.pkl")
pca = joblib.load("pca.pkl")

# Load the mean and standard deviation for feature scaling
X_mean = np.load("X_mean.npy")
X_std = np.load("X_std.npy")

def preprocess_image(img):
    # Convert to grayscale
    img_gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)

    # Apply adaptive histogram equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_equalized = clahe.apply(img_gray)

    # Apply thresholding
    _, img_thresholded = cv2.threshold(img_equalized, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Resize and reshape
    img_resized = cv2.resize(img_thresholded, (28, 28))
    x = img_resized.flatten().reshape(1, -1)

    return x

def recognize_handwriting(img):
    try:
        # Preprocess the user input image
        ##x_scaled = (preprocess_image(img) - X_mean) / X_std
        x_scaled = preprocess_image(img)
        x_pca = pca.transform(x_scaled)
        prediction = sv.predict(x_pca)[0]
        return f"The predicted digit is: {prediction}"
    except Exception as e:
        print(e)  # Optional: print the error message to the console
        return "Character not recognized"

iface = gr.Interface(fn=recognize_handwriting, inputs="sketchpad", outputs="label")
iface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


OpenCV(4.7.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x981fb336::Set<1,-1,-1>,struct cv::impl::A0x981fb336::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1

